In [1]:
import sys
import importlib
sys.path.append('../../')

import lxml.html as html

import knutils.autoscraper as kas

In [41]:
importlib.reload(kas)

<module 'knutils.autoscraper' from '../..\\knutils\\autoscraper.py'>

In [2]:
def make_classifier(root, rel_depth=5, fwditer_num=20):
    depth_stats, tag_stats, attrk_stats, attrv_stats, attrkv_stats = kas.AutoScraper.get_stats(root)

    p_used_tags = ['a','td','img','tr','div','table','script','meta','span','link','input','center','param',
                   'html','head','title','body','form','object','embed']
    t_used_attrk = ['class', 'id', 'name', 'title', 'type']
    t_used_attrv_raw = [z for (k, v) in attrkv_stats.items() if k in t_used_attrk for (z, n) in v]
    t_used_attrv = list(set(t_used_attrv_raw))
    p_used_attrkv = [(k, v) for k in t_used_attrk for v in t_used_attrv if v in [x[0] for x in attrkv_stats[k]]]

    p_context = [('cur', lambda p: p),
                 ('prev-1', lambda p: kas.AutoScraper.get_sibling(p, -1)),
                 ('next-1', lambda p: kas.AutoScraper.get_sibling(p, +1)),
                ('child-0',lambda p: kas.AutoScraper.get_child(p, 0)),
                 ('child-1',lambda p: kas.AutoScraper.get_child(p, 1)),
                 ('child-2',lambda p: kas.AutoScraper.get_child(p, 2))]
    classifier1 = kas.ParentRelClassifier(kas.ContextClassifier(kas.TagClassifier(p_used_tags), p_context), rel_depth)
    classifier2 = kas.ParentRelClassifier(kas.ContextClassifier(kas.AttribClassifier(p_used_attrkv), p_context), rel_depth)
    classifier3 = kas.FwdIterClassifier(kas.TagClassifier(p_used_tags), fwditer_num)
    return kas.AutoScraper({'tags':classifier1, 'attrs':classifier2, 'iter':classifier3})
    
def custom_order(clsf_name, split, split_stat):
    res = kas.SplitStat.calc_gini(clsf_name, split, split_stat)
    
    if clsf_name=='iter':
        if split.base().tag_name() == 'other':
            res *= 0.1
        res *= 0.25
    elif clsf_name=='tags':
        if split.base().base().tag_name()=='other':
            res *= 0.1
            
    if clsf_name in ('tags', 'attrs'):
        if split.base().name() != 'cur':
            res *= 0.1
    
    return res

In [3]:
root = html.parse('../../../DataSets/SiteSample/S2/List.html')

In [4]:
#link to data
target_dscs = []
target_urls = []
for x in root.iter():
    if x.text is not None and 'Вчера' in x.text:
        target_dscs.append(x.getparent())
    if x.tag == 'a' and '#comment' not in x.attrib['href'] and 'newsid' in x.attrib['href'] \
        and 'далее' not in x.text_content():
            target_urls.append(x)

In [5]:
fin_classifier = make_classifier(root)

In [6]:
%%time
dsc_scraper = fin_classifier.fit(root, target_dscs, order_function=custom_order)
url_scraper = fin_classifier.fit(root, target_urls, order_function=custom_order)

Wall time: 9.78 s


In [42]:
d_scrapers = {'dsc': dsc_scraper, 'url': url_scraper}
d_structure = {'data':{'entry':(1,), 'other': (0,0)}, 'entry':{'url':(1,1), 'other':(0,0)}}
fin_scraper = kas.MultiScraper(d_scrapers, d_structure)

In [7]:
url_scraper.get_split_tree()

(('tags', <td> at context cur at rel-depth=2, 555-0 vs 141-10),
 (('tags', <a> at context cur at rel-depth=0, 128-0 vs 13-10),
  (('tags', <div> at context cur at rel-depth=1, 2-10 vs 11-0),
   None,
   (('tags', <span> at context cur at rel-depth=1, 0-10 vs 2-0), None, None)),
  None),
 None)

In [8]:
dsc_scraper.get_split_tree()

(('tags', <table> at context cur at rel-depth=1, 619-0 vs 77-10),
 (('iter', <a> at fwd-iter-id=4, 75-0 vs 2-10),
  (('iter', <a> at fwd-iter-id=2, 2-0 vs 0-10), None, None),
  None),
 None)

In [43]:
res = fin_scraper.parse(root)

hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
hit none in MultiScraper.induce_hierarchy_step - check adequacy
